In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ===================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In this notebook, we will show how we do inference with our trained deep learning recommender model using Triton Inference Server. In this example, we deploy the NVTabular workflow and HugeCTR model with Triton Inference Server. We deploy them as an ensemble. For each request, Triton Inference Server will feed the input data through the NVTabular workflow and its output through the HugeCR model.

As we went through in the previous notebook, [movielens-HugeCTR](https://github.com/NVIDIA/NVTabular/blob/main/examples/inference_triton/inference-HugeCTR/movielens-HugeCTR.ipynb), NVTabular provides a function to save the NVTabular workflow via `export_hugectr_ensemble`. This function does not only save NVTabular workflow, but also saves the trained HugeCTR model and ensemble model to be served to Triton IS.

## Getting Started

Let's import required libraries.

In [2]:
import tritonclient.grpc as httpclient

import cudf
import numpy as np

### Load Models on Triton Server

At this stage, you should launch the Triton Inference Server docker container with the following script:

```
docker run -it --gpus=all -p 8000:8000 -p 8001:8001 -p 8002:8002 -v ${PWD}:/model nvcr.io/nvidia/merlin/merlin-inference:0.5.2
```

After you started the container you can start triton server with the command below:

```
tritonserver --model-repository=path_to_models --backend-config=hugectr,movielens=path_to_json_file --backend-config=hugectr,supportlonglong=true --model-control-mode=explicit 
```

Note: The model-repository path is `/model/models/`. The models haven't been loaded, yet. We can request triton server to load the saved ensemble.  We initialize a triton client. The path for the json file is `/model/models/movielens/1/movielens.json`.

In [5]:
# disable warnings
import warnings

warnings.filterwarnings("ignore")

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


In [7]:
triton_client.is_server_live()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

In [8]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '162'}>
bytearray(b'[{"name":"movielens","version":"1","state":"READY"},{"name":"movielens_ens","version":"1","state":"READY"},{"name":"movielens_nvt","version":"1","state":"READY"}]')


[{'name': 'movielens', 'version': '1', 'state': 'READY'},
 {'name': 'movielens_ens', 'version': '1', 'state': 'READY'},
 {'name': 'movielens_nvt', 'version': '1', 'state': 'READY'}]

Let's load our models to Triton Server.

In [9]:
%%time

triton_client.load_model(model_name="movielens_nvt")

POST /v2/repository/models/movielens_nvt/load, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_nvt'
CPU times: user 1.39 ms, sys: 1.07 ms, total: 2.46 ms
Wall time: 1.68 ms


In [10]:
%%time

triton_client.load_model(model_name="movielens")

POST /v2/repository/models/movielens/load, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens'
CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 1.73 ms


Finally, we load our ensemble model `movielens_ens`.

In [11]:
%%time

triton_client.load_model(model_name="movielens_ens")

POST /v2/repository/models/movielens_ens/load, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_ens'
CPU times: user 2.59 ms, sys: 0 ns, total: 2.59 ms
Wall time: 1.75 ms


Let's send a request to Inference Server and print out the response. Since in our example above we do not have continuous columns, below our only inputs are categorical columns.

In [12]:
from tritonclient.utils import np_to_triton_dtype

model_name = "movielens_ens"
col_names = ["userId", "movieId"]
# read in a batch of data to get transforms for
batch = cudf.read_parquet("/model/data/valid.parquet", num_rows=64)[col_names]
print(batch, "\n")

# convert the batch to a triton inputs
columns = [(col, batch[col]) for col in col_names]
inputs = []

col_dtypes = [np.int64, np.int64]
for i, (name, col) in enumerate(columns):
    d = col.values_host.astype(col_dtypes[i])
    d = d.reshape(len(d), 1)
    inputs.append(httpclient.InferInput(name, d.shape, np_to_triton_dtype(col_dtypes[i])))
    inputs[i].set_data_from_numpy(d)
# placeholder variables for the output
outputs = []
outputs.append(httpclient.InferRequestedOutput("OUTPUT0"))
# make the request
with httpclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(model_name, inputs, request_id=str(1), outputs=outputs)

print("predicted sigmoid result:\n", response.as_numpy("OUTPUT0"))

          userId  movieId
15347762   99476   104374
16647840  107979     2634
23915192  155372     1614
10052313   65225     7153
12214125   79161      500
...          ...      ...
17138306  111072     1625
21326655  138575    81591
5664631    36671     8861
217658      1535   111759
11842246   76766   109487

[64 rows x 2 columns] 

predicted sigmoid result:
 [0.8525418  0.5196209  0.4738261  0.9152796  0.5628397  0.5767327
 0.14926538 0.5918787  0.64609843 0.4327694  0.41364846 0.35985473
 0.6261382  0.7676647  0.9354779  0.65640175 0.50512415 0.9049591
 0.50332576 0.7348553  0.40654865 0.883268   0.6378412  0.6765608
 0.91225433 0.7841664  0.9292699  0.9344907  0.8850527  0.69988066
 0.9261229  0.63722944 0.2109657  0.48579854 0.69964963 0.33116272
 0.10877445 0.2785232  0.8841015  0.64328575 0.8946053  0.8899708
 0.4993149  0.58824956 0.88286483 0.91582847 0.56018364 0.05247024
 0.8787289  0.87412    0.5005526  0.18884465 0.50267965 0.70738006
 0.63269293 0.92454106 0.41982028 0.5